In [1]:
import os
import numpy as np
import json
import pandas as pd
import itertools
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
import re
import matplotlib.pyplot as plt

In [2]:
def create_dataframe(dataset_path, source):
    """Function used to create a Pandas DataFrame containing specifications page titles

    Reads products specifications from the file system ("dataset_path" variable in the main function) and creates a Pandas DataFrame where each row is a
    specification. The columns are 'source' (e.g. www.sourceA.com), 'spec_number' (e.g. 1) and the 'page title'. Note that this script will consider only
    the page title attribute for simplicity.

    Args:
        dataset_path (str): The path to the dataset

    Returns:
        df (pd.DataFrame): The Pandas DataFrame containing specifications and page titles
    """

    print('>>> Creating dataframe...\n')
    columns_df = ['source', 'spec_number', 'spec_id', 'page_title']
    
    df = pd.DataFrame()
    progressive_id = 0
    progressive_id2row_df = {}
    #for source in tqdm(os.listdir(dataset_path)):
    #source = "buy.net"
    for specification in os.listdir(os.path.join(dataset_path, source)):
        specification_number = specification.replace('.json', '')
        specification_id = '{}//{}'.format(source, specification_number)
        with open(os.path.join(dataset_path, source, specification)) as specification_file:
            columns_df = ['source', 'spec_number', 'spec_id']
            specification_data = json.load(specification_file)
            attrs = []
            for k, v in specification_data.items():
                columns_df.append(k)
                attrs.append(v)
            row = [source, specification_number, specification_id]
            row.extend(attrs)
            progressive_id2row_df.update({progressive_id: row})
            progressive_id += 1
            df = df.append(pd.DataFrame.from_dict(progressive_id2row_df, orient='index', columns=columns_df))
            progressive_id2row_df = {}
    #print(df)
    print('>>> Dataframe created successfully!\n')
    return df

In [3]:
df = create_dataframe('../../datasets/unlabeled/2013_camera_specs', "www.pcconnection.com")

>>> Creating dataframe...



/Users/gfotiadis/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


>>> Dataframe created successfully!



In [4]:
df.head()

,35mm equivalent focal length max,35mm equivalent focal length min,<page title>,actual weight,audio input type,battery run time max,battery technology,camera type,color,compatibility notes,...,spec_id,spec_number,special effects,still image format,storage capacity,video input type,warranty labor,warranty parts,warranty details,weight
0,NaN,NaN,"Buy Sony Cyber-shot DSC-WX350 Digital Camera, ...",NaN,NaN,NaN,NaN,Compact,Black,NaN,...,www.pcconnection.com//12423,12423,NaN,NaN,NaN,NaN,Call for Warranty,Call for Warranty,NaN,NaN
1,NaN,NaN,"Buy Nikon COOLPIX A Digital Camera, 16.2MP, Bl...",NaN,NaN,NaN,NaN,Compact,Black,NaN,...,www.pcconnection.com//12198,12198,NaN,NaN,NaN,NaN,Call for Warranty,Call for Warranty,NaN,NaN
2,NaN,NaN,"Buy Pentax K50 Digital SLR Camera 16MP, Black ...",NaN,NaN,NaN,NaN,SLR camera,Black,NaN,...,www.pcconnection.com//12177,12177,NaN,NaN,NaN,NaN,Call for Warranty,Call for Warranty,NaN,NaN
3,NaN,NaN,"Buy Panasonic 16.1MP Digital Camera, 60x Zoom,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,www.pcconnection.com//12435,12435,NaN,NaN,NaN,NaN,Call for Warranty,Call for Warranty,NaN,NaN
4,NaN,NaN,Buy Olympus E-P5 PEN Mirrorless Digital Camera...,NaN,NaN,NaN,NaN,Mirrorless system,Black,NaN,...,www.pcconnection.com//4436,4436,NaN,NaN,NaN,NaN,Call for Warranty,Call for Warranty,NaN,NaN


In [5]:
cols = ["spec_id", "camera type", "display size", "megapixels"]

In [6]:
df = df[cols]

In [7]:
df.head()

,spec_id,camera type,display size,megapixels
0,www.pcconnection.com//12423,Compact,3 in,18.2 megapixel(s)
1,www.pcconnection.com//12198,Compact,3 in,16.2 megapixel(s)
2,www.pcconnection.com//12177,SLR camera,3 in,16.3 megapixel(s)
3,www.pcconnection.com//12435,NaN,NaN,NaN
4,www.pcconnection.com//4436,Mirrorless system,3 in,16.1 megapixel(s)


## Camera type

In [8]:
df["camera type"].value_counts()

Compact                        89
Mirrorless system              45
SLR camera                     35
Point & Shoot camera           16
Smartphone attachable           2
Point & Shoot / Zoom camera     1
Name: camera type, dtype: int64

In [14]:
def clean_camera_type(camera):
    if pd.isna(camera):
        return camera
    else:
        punctuation = "!#$%&'()*+,-./:;<=>?@[\]^_`{|}~€£¥₹₽"
        camera = camera.lower()
        for p in punctuation:
            camera = camera.replace(p, "")
        camera = camera.replace("camera", "")
        camera = camera.replace("zoom", "")
        camera = ' '.join(camera.split())
        return camera

In [15]:
df["camera type"].apply(clean_camera_type).value_counts()

compact                  89
mirrorless system        45
slr                      35
point shoot              17
smartphone attachable     2
Name: camera type, dtype: int64

In [16]:
df["camera type"] = df["camera type"].apply(clean_camera_type)

## Display size

In [18]:
df["display size"].value_counts()

3 in      134
2.7 in     27
3.2 in     14
3.3 in      3
2.5 in      2
4.8 in      2
2.4 in      2
2.8 in      1
3.5 in      1
Name: display size, dtype: int64

In [19]:
def clean_display_size(value):
    if pd.isna(value):
        return value
    else:
        return float(value.split()[0])

In [21]:
df["display size"] = df["display size"].apply(clean_display_size)

In [22]:
df.head()

,spec_id,camera type,display size,megapixels
0,www.pcconnection.com//12423,compact,3.0,18.2 megapixel(s)
1,www.pcconnection.com//12198,compact,3.0,16.2 megapixel(s)
2,www.pcconnection.com//12177,slr,3.0,16.3 megapixel(s)
3,www.pcconnection.com//12435,NaN,NaN,NaN
4,www.pcconnection.com//4436,mirrorless system,3.0,16.1 megapixel(s)


## Megapixels

In [23]:
df["megapixels"].value_counts()

16.1 megapixel(s)     36
16 megapixel(s)       35
16.2 megapixel(s)     10
14.2 megapixel(s)      9
18 megapixel(s)        9
20.1 megapixel(s)      8
18.2 megapixel(s)      8
12 megapixel(s)        7
24.3 megapixel(s)      6
12.1 megapixel(s)      6
24.2 megapixel(s)      5
20.2 megapixel(s)      4
20 megapixel(s)        4
16.4 megapixel(s)      4
10.1 megapixel(s)      4
16.3 megapixel(s)      4
24.1 megapixel(s)      4
13.2 megapixel(s)      3
20.4 megapixel(s)      2
12.2 megapixel(s)      2
36.3 megapixel(s)      2
12.4 megapixel(s)      2
22.3 megapixel(s)      2
23.35 megapixel(s)     2
18.4 megapixel(s)      1
20.9 megapixel(s)      1
20.3 megapixel(s)      1
14.4 megapixel(s)      1
14.14 megapixel(s)     1
14 megapixel(s)        1
36.4 megapixel(s)      1
12.3 megapixel(s)      1
9.1 megapixel(s)       1
Name: megapixels, dtype: int64

In [24]:
def clean_megapixels(value):
    if pd.isna(value):
        return value
    else:
        return round(float(value.split()[0]), 1)

In [26]:
df["megapixels"] = df["megapixels"].apply(clean_megapixels)

In [27]:
df.head()

,spec_id,camera type,display size,megapixels
0,www.pcconnection.com//12423,compact,3.0,18.2
1,www.pcconnection.com//12198,compact,3.0,16.2
2,www.pcconnection.com//12177,slr,3.0,16.3
3,www.pcconnection.com//12435,NaN,NaN,NaN
4,www.pcconnection.com//4436,mirrorless system,3.0,16.1


## Saving

In [28]:
df.to_csv("../../datasets/unlabeled/cleaned/pcconnection.csv", index=False)